# Analysis app_events (+labels, categories)

In [1]:
import os
import math

import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

%matplotlib inline

dir_in = './data_ori/'

In [2]:
# Load app_events
print('Reading app categories...')
app_cat = pd.read_csv(dir_in + 'label_categories.csv')

# Event ID is unique integer
print('Reading app labels...')
app_labels=pd.read_csv(dir_in + 'app_labels.csv')

app_labels = app_labels.merge(app_cat, how='left', on='label_id')
app_labels['ones'] = 1

appcats = app_labels.pivot_table(values='ones',columns='category',index=['app_id'])
appcats.fillna(0,inplace=True)

Reading app categories...
Reading app labels...


In [3]:
appcats.head()

category,1 free,1 reputation,1 vitality,3 kindom game,80s Japanese comic,90s Japanese comic,A beauty care,A shares,ARPG,Academic Information,...,travel,tribe,trickery,unknown,video,violence comic,vitality,war chess,weibo,zombies game
app_id,,,,,,,,,,,,,,,,,,,,,
-9223281467940916832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9222877069545393219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9222785464897897681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9222198347540756780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9221970424041518544,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
print('Top 15 categories:')
appcats.sum().sort_values(ascending=False).head(15)

Top 15 categories:


category
Industry tag                56902.0
Custom label                53936.0
Tencent                     49320.0
game                        48707.0
Property Industry 2.0       45697.0
1 free                      19083.0
Services 1                  11840.0
Property Industry new        9955.0
Relatives 1                  9027.0
Irritation / Fun 1           8831.0
Cards RPG                    7375.0
Casual puzzle categories     7052.0
Personal Effectiveness 1     5910.0
ARPG                         5288.0
Chess categories             5135.0
dtype: float64

In [40]:
print('Reading app events...')
app_events = pd.read_csv(dir_in + 'app_events.csv', index_col='app_id', nrows=100000)

Reading app events...


In [41]:
print('Shape app_events before merge:', app_events.shape)
# Too big to merge, completely, let start analysis on small subsample
app_events = pd.merge(app_events,appcats,how='inner',left_index=True, right_index=True)
print('Shape app_events after merge:', app_events.shape)

('Shape app_events before merge:', (100000, 3))
('Shape app_events after merge:', (100000, 476))


In [42]:
print('Events withs most installed apps')
app_events.reset_index().groupby(['event_id']).is_installed.sum().sort_values(ascending=False).head(5)

Events withs most installed apps


event_id
5140    232
2359    171
6974    167
123     153
4853    146
Name: is_installed, dtype: int64

In [43]:
print('Events with most active apps')
app_events.reset_index().groupby(['event_id']).is_active.sum().sort_values(ascending=False).head(5)

Events with most active apps


event_id
3685    41
2831    38
5849    37
5964    37
3849    36
Name: is_active, dtype: int64

In [44]:
print('Installed and active compared')
app_events.reset_index().groupby(['event_id'])['is_installed','is_active'].sum().head(5)

Installed and active compared


,is_installed,is_active
event_id,,
2,19,6
6,38,15
7,40,13
9,35,7
16,31,1


In [45]:
print('Largest percentage active')
df = app_events.reset_index().groupby(['event_id'])['is_installed','is_active'].sum()
print (df.is_active / df.is_installed).sort_values(ascending=False).head(10)
print('Largest percentage inactive') # Might be predictor for age?
print (df.is_installed / df.is_active).sort_values(ascending=False).head(10)

Largest percentage active
event_id
4027    1.0
4965    1.0
2026    1.0
2012    1.0
4989    1.0
4995    1.0
4999    1.0
2009    1.0
5020    1.0
2007    1.0
dtype: float64
Largest percentage inactive
event_id
2359    inf
6787    inf
6621    inf
1851    inf
114     inf
6519    inf
4339    inf
6414    inf
2254    inf
2653    inf
dtype: float64


In [46]:
app_cat_col=appcats.columns

In [47]:
app_events.reset_index().groupby(['event_id'])[app_cat_col].sum().max(axis=1).head(5)

event_id
2     16.0
6     24.0
7     30.0
9     26.0
16    23.0
dtype: float64

Preferably four different counts:
1. per category sum(is_installed) totals
2. per category sum(is_installed) relative to total installed
3. per category sum(is_active) totals
4. per category sum(is_active) relative to total_active

In [48]:
# 1:
df1 = app_events.reset_index().groupby(['event_id']).sum()
df1.head(5)

,app_id,is_installed,is_active,1 free,1 reputation,1 vitality,3 kindom game,80s Japanese comic,90s Japanese comic,A beauty care,...,travel,tribe,trickery,unknown,video,violence comic,vitality,war chess,weibo,zombies game
event_id,,,,,,,,,,,,,,,,,,,,,
2,2.805985e+19,19.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3.039371e+19,38.0,15.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0
7,2.623446e+19,40.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0
9,2.739554e+19,35.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0
16,2.464116e+19,31.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,3.0,0.0,0.0,0.0,2.0,0.0


In [49]:
# 2:
df2 = app_events.reset_index().groupby(['event_id']).sum()
df2[app_cat_col] = df2[app_cat_col].div(df['is_installed'], axis=0)
df2.head(5)

,app_id,is_installed,is_active,1 free,1 reputation,1 vitality,3 kindom game,80s Japanese comic,90s Japanese comic,A beauty care,...,travel,tribe,trickery,unknown,video,violence comic,vitality,war chess,weibo,zombies game
event_id,,,,,,,,,,,,,,,,,,,,,
2,2.805985e+19,19.0,6.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.052632,0.0,0.0,0.263158,0.000000,0.0,0.0,0.0,0.000000,0.0
6,3.039371e+19,38.0,15.0,0.078947,0.026316,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.236842,0.052632,0.0,0.0,0.0,0.000000,0.0
7,2.623446e+19,40.0,13.0,0.050000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.125000,0.025000,0.0,0.0,0.0,0.025000,0.0
9,2.739554e+19,35.0,7.0,0.085714,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.171429,0.085714,0.0,0.0,0.0,0.000000,0.0
16,2.464116e+19,31.0,1.0,0.161290,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.258065,0.096774,0.0,0.0,0.0,0.064516,0.0


In [50]:
# 3:
df3 = app_events[app_events.is_active==1].reset_index().groupby(['event_id']).sum()
df3.head(5)

,app_id,is_installed,is_active,1 free,1 reputation,1 vitality,3 kindom game,80s Japanese comic,90s Japanese comic,A beauty care,...,travel,tribe,trickery,unknown,video,violence comic,vitality,war chess,weibo,zombies game
event_id,,,,,,,,,,,,,,,,,,,,,
2,3.140823e+19,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.696323e+19,15.0,15.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
7,-7.023240e+18,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
9,2.587383e+19,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,5.516228e+18,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# 4:
df4 = app_events[app_events.is_active==1].reset_index().groupby(['event_id']).sum()
df4[app_cat_col] = df4[app_cat_col].div(df['is_active'], axis=0)
df4.head(5)

,app_id,is_installed,is_active,1 free,1 reputation,1 vitality,3 kindom game,80s Japanese comic,90s Japanese comic,A beauty care,...,travel,tribe,trickery,unknown,video,violence comic,vitality,war chess,weibo,zombies game
event_id,,,,,,,,,,,,,,,,,,,,,
2,3.140823e+19,6.0,6.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
6,2.696323e+19,15.0,15.0,0.066667,0.066667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.200000,0.066667,0.0,0.0,0.0,0.000000,0.0
7,-7.023240e+18,13.0,13.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.153846,0.000000,0.0,0.0,0.0,0.076923,0.0
9,2.587383e+19,7.0,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
16,5.516228e+18,1.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0


## Merge with devices

In [66]:
print('Loading train and event data')
train = pd.read_csv(dir_in + 'gender_age_train.csv', dtype={'device_id':np.str})
event = pd.read_csv(dir_in + 'events.csv', dtype={'device_id':np.str})

Loading train and event data


In [67]:
print('Size event:', df1.shape)
print('Size df1:', event.shape)
print('Size train:', train.shape)
event_app = pd.merge(event, df1, how='inner', left_on='event_id',right_index=True)
event_app = pd.merge(event_app, train, how='inner', on='device_id')
print('Size after merge:', event_app.shape)

('Size event:', (3553, 476))
('Size df1:', (3252950, 5))
('Size train:', (74645, 4))
('Size after merge:', (1336, 484))


In [68]:
event_app.groupby('device_id').size().sort_values(ascending=False).head()

device_id
3074308677943390456     21
1057289835566390654      5
-4483235488687461351     4
2526025498254449332      4
4657697113512107583      3
dtype: int64

In [88]:
summary = event_app.groupby(['gender','device_id']).mean().unstack('gender')
summary.drop(['event_id','longitude','latitude','app_id'],axis=1, inplace=True)
print summary.head()

                     is_installed       is_active       1 free       \
gender                          F     M         F     M      F    M   
device_id                                                             
-1032555705119615296          9.0   NaN       9.0   NaN    0.0  NaN   
-1037012685448076071          NaN  63.0       NaN   2.0    NaN  4.0   
-1084174363886138500          NaN  20.5       NaN   9.5    NaN  1.0   
-1086079967197442674          NaN   8.0       NaN   8.0    NaN  0.5   
-1104601348418161884          NaN  56.5       NaN  20.5    NaN  5.0   

                     1 reputation      1 vitality       ...  vitality       \
gender                          F    M          F    M  ...         F    M   
device_id                                               ...                  
-1032555705119615296          0.0  NaN        0.0  NaN  ...       0.0  NaN   
-1037012685448076071          NaN  0.0        NaN  0.0  ...       NaN  0.0   
-1084174363886138500          NaN  0.5   

In [96]:
is_ins = summary.is_installed.sum()
is_act = summary.is_active.sum()

print is_ins / max(is_ins)
print is_act / max(is_act)

gender
F    0.279251
M    1.000000
dtype: float64
gender
F    0.381879
M    1.000000
dtype: float64


In [128]:
summary.sum().unstack('gender').sort_values('F',ascending=False).head()

gender,F,M
age,10832.000000,28758.000000
is_installed,7349.750000,26319.533333
Industry tag,5495.083333,19446.450000
Property Industry 2.0,3879.750000,13542.950000
Property Industry 1.0,3121.166667,10847.666667
